In [1]:
import pandas as pd
import seaborn as sns
import numpy as np

from lxml import html
import requests
import codecs

from itertools import chain
import re
import string
import unidecode
import nltk
from nltk.corpus import stopwords  
from nltk.tokenize import word_tokenize, sent_tokenize

import Levenshtein
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
import gensim

In [2]:
pagina = 'https://dreguera.github.io/'

In [3]:
page = requests.get(pagina)
tree = html.fromstring(page.content)

In [4]:
asunto = tree.xpath("//div[@id='ariketa2']/div[@class='post']/div/span[@id='subject']/text()")
autor=tree.xpath("//div[@id='ariketa2']/div[@class='post']/div/span[@id='author']/text()")
content= tree.xpath("//div[@id='ariketa2']/div[@class='post']/div/div[@id='content']/text()")

In [9]:
de="Dani"
result=[]

for m in autor:
    a = m.split(' ')[0]
    if Levenshtein.distance(a,de)==0:
        result.append(a)
print(len(result))

2


In [31]:
content[0]

'He estado corrigiendo los trabajos de Data Science y la parte de aprendizaje no supervisado la han realizado bastante bien, Â¿QuÃ© algoritmos de clustering has dado en clase?'

In [33]:
content[2]

'MeanShift para clustering por densidad habeis visto? Yo suelo utilizar para datos numÃ©ricos y me da muy buenos resultados para descartar outliers'

# a

In [37]:
file_docs=[]
file_docs2=[]
tokens = sent_tokenize(content[0])
for line in tokens:
    file_docs.append(line)
    
tokens = sent_tokenize(content[2])
for line in tokens:
    file_docs2.append(line)

In [38]:
gen_docs = [[w.lower() for w in word_tokenize(text)] 
            for text in file_docs]

In [39]:
gen_docs2 = [[w.lower() for w in word_tokenize(text)] 
            for text in file_docs2]

In [40]:
dictionary = gensim.corpora.Dictionary(gen_docs)
print(dictionary.token2id)

{',': 0, '?': 1, 'algoritmos': 2, 'aprendizaje': 3, 'bastante': 4, 'bien': 5, 'clase': 6, 'clustering': 7, 'corrigiendo': 8, 'dado': 9, 'data': 10, 'de': 11, 'en': 12, 'estado': 13, 'han': 14, 'has': 15, 'he': 16, 'la': 17, 'los': 18, 'no': 19, 'parte': 20, 'realizado': 21, 'science': 22, 'supervisado': 23, 'trabajos': 24, 'y': 25, 'â¿quã©': 26}


In [41]:
corpus = [dictionary.doc2bow(gen_doc) for gen_doc in gen_docs]

In [42]:
tf_idf = gensim.models.TfidfModel(corpus)

In [43]:
for doc in tf_idf[corpus]:
    print([[dictionary[id], np.around(freq, decimals=2)] for id, freq in doc])

[]


In [44]:
sims = gensim.similarities.Similarity('a',tf_idf[corpus],
                                        num_features=len(dictionary))

In [45]:
for line in file_docs2:
    query_doc = [w.lower() for w in word_tokenize(line)]
    query_doc_bow = dictionary.doc2bow(query_doc)

In [46]:
query_doc_tf_idf = tf_idf[query_doc_bow]

In [47]:
print('Comparing Result:', sims[query_doc_tf_idf]) 

Comparing Result: [0.]


In [48]:
import numpy as np

sum_of_sims =(np.sum(sims[query_doc_tf_idf], dtype=np.float32))
print(sum_of_sims)

0.0


In [49]:
percentage_of_similarity = round(float((sum_of_sims / len(file_docs)) * 100))
print(f'Average similarity float: {float(sum_of_sims / len(file_docs))}')
print(f'Average similarity percentage: {float(sum_of_sims / len(file_docs)) * 100}')
print(f'Average similarity rounded percentage: {percentage_of_similarity}')

Average similarity float: 0.0
Average similarity percentage: 0.0
Average similarity rounded percentage: 0


# b

In [63]:
file_docs=[]
file_docs2=[]
tokens = sent_tokenize(content[1])
for line in tokens:
    file_docs.append(line)
    
tokens = sent_tokenize(content[3])
for line in tokens:
    file_docs2.append(line)

In [64]:
gen_docs = [[w.lower() for w in word_tokenize(text)] 
            for text in file_docs]

In [65]:
gen_docs2 = [[w.lower() for w in word_tokenize(text)] 
            for text in file_docs2]

In [66]:
dictionary = gensim.corpora.Dictionary(gen_docs)
print(dictionary.token2id)

{',': 0, '.': 1, 'centrado': 2, 'clustering': 3, 'dani': 4, 'de': 5, 'densidad': 6, 'el': 7, 'en': 8, 'he': 9, 'hola': 10, 'la': 11, 'me': 12, 'no': 13, 'parte': 14, 'por': 15, 'sobre': 16, 'supervisado': 17, 'todo': 18, 'dbscan': 19, 'hdscan': 20, 'hemos': 21, 'optics': 22, 'visto': 23, 'y': 24}


In [67]:
corpus = [dictionary.doc2bow(gen_doc) for gen_doc in gen_docs]

In [68]:
tf_idf = gensim.models.TfidfModel(corpus)

In [69]:
for doc in tf_idf[corpus]:
    print([[dictionary[id], np.around(freq, decimals=2)] for id, freq in doc])

[['.', 0.22], ['centrado', 0.22], ['clustering', 0.22], ['dani', 0.22], ['de', 0.22], ['densidad', 0.22], ['el', 0.22], ['en', 0.44], ['he', 0.22], ['hola', 0.22], ['la', 0.22], ['me', 0.22], ['no', 0.22], ['parte', 0.22], ['por', 0.22], ['sobre', 0.22], ['supervisado', 0.22], ['todo', 0.22]]
[['dbscan', 0.41], ['hdscan', 0.41], ['hemos', 0.41], ['optics', 0.41], ['visto', 0.41], ['y', 0.41]]


In [70]:
sims = gensim.similarities.Similarity('b',tf_idf[corpus],
                                        num_features=len(dictionary))

In [71]:
for line in file_docs2:
    query_doc = [w.lower() for w in word_tokenize(line)]
    query_doc_bow = dictionary.doc2bow(query_doc)

In [72]:
query_doc_tf_idf = tf_idf[query_doc_bow]

In [73]:
print('Comparing Result:', sims[query_doc_tf_idf]) 

Comparing Result: [0.19518001 0.5477226 ]


In [74]:
import numpy as np

sum_of_sims =(np.sum(sims[query_doc_tf_idf], dtype=np.float32))
print(sum_of_sims)

0.7429026


In [75]:
percentage_of_similarity = round(float((sum_of_sims / len(file_docs)) * 100))
print(f'Average similarity float: {float(sum_of_sims / len(file_docs))}')
print(f'Average similarity percentage: {float(sum_of_sims / len(file_docs)) * 100}')
print(f'Average similarity rounded percentage: {percentage_of_similarity}')

Average similarity float: 0.3714512884616852
Average similarity percentage: 37.14512884616852
Average similarity rounded percentage: 37


# c

In [86]:
file_docs=[]
file_docs2=[]
tokens = sent_tokenize(content[1])
for line in tokens:
    file_docs.append(line)
    
tokens = sent_tokenize(content[4])
for line in tokens:
    file_docs2.append(line)

In [87]:
gen_docs = [[w.lower() for w in word_tokenize(text)] 
            for text in file_docs]

In [88]:
gen_docs2 = [[w.lower() for w in word_tokenize(text)] 
            for text in file_docs2]

In [89]:
dictionary = gensim.corpora.Dictionary(gen_docs)
print(dictionary.token2id)

{',': 0, '.': 1, 'centrado': 2, 'clustering': 3, 'dani': 4, 'de': 5, 'densidad': 6, 'el': 7, 'en': 8, 'he': 9, 'hola': 10, 'la': 11, 'me': 12, 'no': 13, 'parte': 14, 'por': 15, 'sobre': 16, 'supervisado': 17, 'todo': 18, 'dbscan': 19, 'hdscan': 20, 'hemos': 21, 'optics': 22, 'visto': 23, 'y': 24}


In [90]:
corpus = [dictionary.doc2bow(gen_doc) for gen_doc in gen_docs]

In [91]:
tf_idf = gensim.models.TfidfModel(corpus)

In [92]:
for doc in tf_idf[corpus]:
    print([[dictionary[id], np.around(freq, decimals=2)] for id, freq in doc])

[['.', 0.22], ['centrado', 0.22], ['clustering', 0.22], ['dani', 0.22], ['de', 0.22], ['densidad', 0.22], ['el', 0.22], ['en', 0.44], ['he', 0.22], ['hola', 0.22], ['la', 0.22], ['me', 0.22], ['no', 0.22], ['parte', 0.22], ['por', 0.22], ['sobre', 0.22], ['supervisado', 0.22], ['todo', 0.22]]
[['dbscan', 0.41], ['hdscan', 0.41], ['hemos', 0.41], ['optics', 0.41], ['visto', 0.41], ['y', 0.41]]


In [93]:
sims = gensim.similarities.Similarity('c',tf_idf[corpus],
                                        num_features=len(dictionary))

In [94]:
for line in file_docs2:
    query_doc = [w.lower() for w in word_tokenize(line)]
    query_doc_bow = dictionary.doc2bow(query_doc)

In [95]:
query_doc_tf_idf = tf_idf[query_doc_bow]

In [96]:
print('Comparing Result:', sims[query_doc_tf_idf]) 

Comparing Result: [0.5819144 0.       ]


In [97]:
import numpy as np

sum_of_sims =(np.sum(sims[query_doc_tf_idf], dtype=np.float32))
print(sum_of_sims)

0.5819144


In [98]:
percentage_of_similarity = round(float((sum_of_sims / len(file_docs)) * 100))
print(f'Average similarity float: {float(sum_of_sims / len(file_docs))}')
print(f'Average similarity percentage: {float(sum_of_sims / len(file_docs)) * 100}')
print(f'Average similarity rounded percentage: {percentage_of_similarity}')

Average similarity float: 0.2909572124481201
Average similarity percentage: 29.09572124481201
Average similarity rounded percentage: 29


# D

In [156]:
content1= tree.xpath("//div[@id='ariketa2']/div[@class='post']/div/div[@id='content']")

In [158]:
mensajes_lista = []
for m in content1:
    mensajes_lista.append(m.text)

In [160]:
vectorizer = CountVectorizer().fit_transform(mensajes_lista)
vectors = vectorizer.toarray()

In [161]:
vectors

array([[0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 3,
        0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 2,
        0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0,
        0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1,
        0, 1, 0, 1, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1,
        0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0,
        0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0,
        0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 3, 0, 0, 0, 0, 0, 0, 1, 0,
        0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
        1, 0, 1, 1, 0],
       [1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,

In [162]:
csim=cosine_similarity(vectors)

In [163]:
csim

array([[1.        , 0.40567404, 0.03194383, 0.16574839, 0.07377111,
        0.13093073, 0.        , 0.        ],
       [0.40567404, 1.        , 0.18898224, 0.15689291, 0.34914862,
        0.05163978, 0.06030227, 0.        ],
       [0.03194383, 0.18898224, 1.        , 0.1111874 , 0.12371791,
        0.14638501, 0.05698029, 0.        ],
       [0.16574839, 0.15689291, 0.1111874 , 1.        , 0.12838815,
        0.15191091, 0.        , 0.        ],
       [0.07377111, 0.34914862, 0.12371791, 0.12838815, 1.        ,
        0.        , 0.32897585, 0.        ],
       [0.13093073, 0.05163978, 0.14638501, 0.15191091, 0.        ,
        1.        , 0.        , 0.        ],
       [0.        , 0.06030227, 0.05698029, 0.        , 0.32897585,
        0.        , 1.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 1.        ]])

#ejercicio 2

In [132]:
df = pd.read_csv("Spotify-2000.csv")

In [133]:
df.head()

,Index,Title,Artist,Top Genre,Year,Beats Per Minute (BPM),Energy,Danceability,Loudness (dB),Liveness,Valence,Length (Duration),Acousticness,Speechiness,Popularity
0,1,Sunrise,Norah Jones,adult standards,2004,157,30,53,-14,11,68,201,94,3,71
1,2,Black Night,Deep Purple,album rock,2000,135,79,50,-11,17,81,207,17,7,39
2,3,Clint Eastwood,Gorillaz,alternative hip hop,2001,168,69,66,-9,7,52,341,2,17,69
3,4,The Pretender,Foo Fighters,alternative metal,2007,173,96,43,-4,3,37,269,0,4,76
4,5,Waitin' On A Sunny Day,Bruce Springsteen,classic rock,2002,106,82,58,-5,10,87,256,1,3,59


In [134]:
df[['Artist', 'Top Genre']] = df[['Artist', 'Top Genre']].fillna('')

In [135]:
def combined_features(row):
    return row['Artist']+" "+row['Top Genre']
df["combined_features"] = df.apply(combined_features, axis =1)

In [136]:
vectorizer = CountVectorizer().fit_transform(df["combined_features"])
vectors = vectorizer.toarray()
csim = cosine_similarity(vectors)

In [137]:
csim

array([[1.  , 0.  , 0.  , ..., 0.  , 0.  , 0.5 ],
       [0.  , 1.  , 0.  , ..., 0.25, 0.  , 0.  ],
       [0.  , 0.  , 1.  , ..., 0.  , 0.  , 0.  ],
       ...,
       [0.  , 0.25, 0.  , ..., 1.  , 0.  , 0.  ],
       [0.  , 0.  , 0.  , ..., 0.  , 1.  , 0.  ],
       [0.5 , 0.  , 0.  , ..., 0.  , 0.  , 1.  ]])

In [138]:
color_pelicula= ['Without Me']

In [139]:
color_pelicula

['Without Me']

In [140]:
type(color_pelicula)

list

In [141]:
def get_index_from_title(Title):
    return df[df.Title == Title]["Index"].values[0]

In [142]:
movie_index = get_index_from_title(color_pelicula[0])
movie_index

10

In [143]:
similar_movies = list(enumerate(csim[movie_index])) 

In [146]:
sorted_similar_movies = sorted(similar_movies, key=lambda x:x[1], reverse=True)
sorted_similar_movies

[(10, 1.0),
 (144, 1.0),
 (387, 1.0),
 (664, 1.0),
 (918, 1.0),
 (928, 1.0),
 (948, 1.0),
 (1686, 1.0),
 (1828, 1.0),
 (1836, 1.0),
 (1909, 1.0),
 (1950, 1.0),
 (1954, 1.0),
 (1985, 1.0),
 (1986, 1.0),
 (1989, 1.0),
 (1990, 1.0),
 (211, 0.5773502691896258),
 (829, 0.5773502691896258),
 (834, 0.5773502691896258),
 (899, 0.5773502691896258),
 (1007, 0.5773502691896258),
 (0, 0.5),
 (33, 0.5),
 (48, 0.5),
 (124, 0.5),
 (138, 0.5),
 (167, 0.5),
 (218, 0.5),
 (230, 0.5),
 (267, 0.5),
 (275, 0.5),
 (315, 0.5),
 (426, 0.5),
 (476, 0.5),
 (627, 0.5),
 (716, 0.5),
 (827, 0.5),
 (843, 0.5),
 (847, 0.5),
 (853, 0.5),
 (858, 0.5),
 (860, 0.5),
 (866, 0.5),
 (870, 0.5),
 (889, 0.5),
 (895, 0.5),
 (907, 0.5),
 (921, 0.5),
 (934, 0.5),
 (945, 0.5),
 (953, 0.5),
 (954, 0.5),
 (960, 0.5),
 (962, 0.5),
 (980, 0.5),
 (982, 0.5),
 (990, 0.5),
 (1016, 0.5),
 (1017, 0.5),
 (1021, 0.5),
 (1029, 0.5),
 (1049, 0.5),
 (1110, 0.5),
 (1115, 0.5),
 (1124, 0.5),
 (1125, 0.5),
 (1132, 0.5),
 (1140, 0.5),
 (1146, 0.5

In [147]:
def get_title_from_index(Index):
    return df[df.Index == Index]["Title"].values[0]

In [149]:
for movie in similar_movies[1:6]:
    print(get_title_from_index(movie[0]))

Sunrise
Black Night
Clint Eastwood
The Pretender
Waitin' On A Sunny Day


In [152]:
def get_title_from_index1(Index):
    return df[df.Index == Index]["Artist"].values[0]

In [153]:
for movie in similar_movies[1:6]:
    print(get_title_from_index1(movie[0]))

Norah Jones
Deep Purple
Gorillaz
Foo Fighters
Bruce Springsteen
